# Exercise 4
## Map making
*** 

### Questions
How can I plot a map, showing the locations of some samples/data?

### Objectives
<div class=obj>
<ol>
    <li>Explore the syntax for plotting a simple map using Cartopy.</li>
    <li>Use more advanced syntax for plotting a local area with shapefile overlays.</li>
    <li>Learn how to loop over files to readin multiple files to Python</li>
    <li>See regular expressions in action to separate files with separate chunks of data.</li>
    <li>Learn to plot spatial data onto a map.</li>
</ol>
    
<ul>
Revise:
    <li>Data readin;</li>
    <li>Data plotting;</li>
    <li>Defining functions.</li>
</ul>
</div>

### Independent coding
Make a map of the continental US's 2019 earthquakes.

<div class=warn>
    <b>Health warning:</b>
    <p></p>
    Plotting maps in Python is just hard.  It's a bit awkward, and the syntax is not particularly intuitive.  Don't worry if you struggle with this.  The exercise does however use lots of elements of coding you have already covered, so it is useful to see these in a new context, even if the map plotting commands are hard to remember.
</div>

## 4.1 Make a simple map
***

Plotting spatial data is a common task in the Geosciences.  There are several tools available in Python to aid with this, including a spatial version of Pandas __[GeoPandas](http://geopandas.org)__, and a high-level interface for making maps __[Geoplot](https://residentmario.github.io/geoplot/index.html)__.  However, here we are going to use __[Cartopy](https://scitools.org.uk/cartopy/docs/v0.15/index.html)__, which is the machinery underlying geoplot.

Let's begin by plottig the globe.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtckr
#this is the import for cartopy, and we are importing its coordinate reference system tool as ccrs
# the coordinate reference system tells us what type map projection we are making
# i.e., we need some mathematical transformation to tell us how to convert the surface of a sphere into a flat plot.
import cartopy.crs as ccrs

#let's set up a figure to plot into, in a similar way to what we have done previously
# we are just specififying the size up front here
fg = plt.figure(figsize=(12, 6))

#now, we add a subplot to our figure, i.e., place a panel within the figure
# the '1,1,1' just indicates we have one subplot
# with ccrs.XXXXX we are adding the map projection, here we use the well known Mercator projection
ax = fg.add_subplot(1, 1, 1,
                         projection=ccrs.Mercator())

#let's add coastlines and specify the resolution we want
# note: we have little choice of resolution, 110m (1:110e6 scale) is the coarsest we can get, we would have to load our
#       coastline file to change this.
ax.coastlines(resolution='110m')

#we can add a 'stock' background image just to make things look nicer.
ax.stock_img();

#and we can place some gridlines on
gl = ax.gridlines(draw_labels=True, linestyle=':', color='k');
#...and let's limit which sides the labels plot on
gl.xlabels_top = False
gl.ylabels_right = False
#finally, placing the latitude markers every 30 degrees
gl.ylocator = mtckr.FixedLocator([-90, -60, -30, 0, 30, 60, 90])

Note, that the figure above nicely demonstrates the problematic distortions that creep in when plotting the whole globe with a Mercator projection: Greenland and the British Isles grow at the expense of equatorial landmasses and
>the Former Soviet Union looks much bigger than Africa or South America. One may wonder whether this illusion has had any influence on U.S. foreign policy.
 <p></p>\- _GMT Manual pages_

These distortions occur because the Mercator projection is __[not an equal-area projection](https://en.wikipedia.org/wiki/Map_projection#Equal-area)__. If you are interested in reading more about the cultural significance of map projections __[here is an article](https://www.bostonglobe.com/metro/2017/03/16/north-america-really-bigger-than-africa-this-map-sets-things-straight/lK52K7aKYFpQ3b8ujJj6LP/story.html)__ discussing some of the implications of map projections in education. 

Let's now try a more sensible global projection that does preserve area, albeit at the expense of other distortions, the __[Mollweide](https://en.wikipedia.org/wiki/Mollweide_projection)__ projection.

In [ ]:
#same as before
fg = plt.figure(figsize=(12, 6))

#now we choose the Molleweide projection
ax = fg.add_subplot(1, 1, 1,
                         projection=ccrs.Mollweide())

#let's add coastlines and specify the resolution we want
# note: we have little choice of resolution, 110m is the coarsest we can get, we would have to load our
#       coastline file to change this.
ax.coastlines(resolution='110m')

#we can add a 'stock' background image just to make things look nicer.
ax.stock_img();

#and we can place some gridlines on
gl = ax.gridlines(draw_labels=False, linestyle=':', color='k');
#note, that cartopy does not currently support plotting long lat labels for the Molleweide projection

This looks much better; the UK is now restored as a small island on the edge of Europe.  

Another potential annoyance of the maps above is that whilst they get all contintal area into the picture, they chop the Pacific in half.  This can easily be changed by specifying the central longitude.

In [ ]:
#same as before
fg = plt.figure(figsize=(12, 6))

#notice now that we are passing the Mollweide project a 'central_longitude' argument
ax = fg.add_subplot(1, 1, 1,
                         projection=ccrs.Mollweide(central_longitude=-150))

#..same as before
ax.coastlines(resolution='110m')
ax.stock_img();
gl = ax.gridlines(draw_labels=False, linestyle=':', color='k');

Note, that map plotting in python is still at a somewhat nascent stage.  There are other powerful tools to plot maps that are currently available outside the Python environment, the best of which is __[GMT](https://github.com/GenericMappingTools)__, but this is beyond the scope of these practicals.  Let's persevere with Python for now!

## 4.2 Make a more advanced map
***

### 4.2.1 Getting started with the map

Let's now aim to plot some data onto a map.  We are going to revisit the dataset of Icelandic eruptions we investigated in __[Exercise 2](Exercise2.ipynb)__.  

First however, we should focus on producing a map of Iceland and plotting on one of its key features: glaciers.  

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtckr
import cartopy as ctp
import cartopy.crs as ccrs

fg = plt.figure(figsize=(12, 6))

#we will use Mercator for this, as for a small region of the globe the area distortion will not be significant
# we need to shift the central longitude onto Iceland
ax = fg.add_subplot(1, 1, 1,
                         projection=ccrs.Mercator(central_longitude=-15))
#we also want to limit the plot area to Iceland
ax.set_extent([-25, -13, 63, 67.], crs=ccrs.Geodetic())

#let's add coastlines and specify the resolution we want
# note: we now want higher resolution, and the highest available to us is 1:10m.  
# we are also going to have to setup the 'feature' to be plotted and then specifically plot that 
#first setup the coastlines
feature = ctp.feature.NaturalEarthFeature(
    name='coastline', category='physical',
    scale='10m',
    edgecolor='#000000', facecolor='lightgrey')
#...then plot
ax.add_feature(feature)

#and we can place some gridlines on, this time I am placing them behind the landmass
# using 'zorder' to indicate where I want the element to appear in the vertical 'stack' of things going onto the plot
gl = ax.gridlines(draw_labels=True, linestyle=':', color='k', zorder=-1);
#...and let's limit which sides the labels plot on
gl.xlabels_top = False
gl.ylabels_right = False

Notice one important feature of how we communicated the 'coordinate reference system' to Cartopy: whenever we provided values that were 'raw' latitudes and longitudes (which we did to constrain the plot extent) we specified that the coordinate reference system was `ccrs.Geodetic()`.  The other option that achieves nearly identical results is to use `ccrs.PlateCarree()`.  You will see that we will need to use both projection types to plot lat-lon data.

### 4.2.2 Plotting on icesheets: shapefile overlays and reading in multiple files

Now, Iceland wouldn't be Iceland without its icesheets.  To plot on the icesheets we need to loop over the several input files inside `data/jokull/` that contain the xy coordinates of the icesheets, and then transform these into an object that can be plotted (Jokull means ice in Icelandic).  This latter task we perform using the `shapely` library (more on this powerful library __[here](https://shapely.readthedocs.io/en/stable/manual.html)__).  Remember to look inside those input files to convince yourself something sensible is going to happen when we try and plot them.

Let's write a function to do the readin and conversion to shape files so that we can call it each time we remake the plot.

In [ ]:
#we are going to need to access files, so we need a tool that can tell us about 
# files on the computer, for this we use 'os'
# the name is already short though, so we aren't going to need to 'import as'!
import os
#Pandas make everything better
import pandas as pd

#we import the Polygon tool from Shapely's geometry library
# we are going to convert the long-lat points of the icesheets into 'polygon' objects that 
# can be plotted nicely on our map
from shapely.geometry import Polygon

#define our function
def poly_jokull():

    #we loop over all files in 'data/jokull', 'files' is going to be the list where we store
    # all file locations
    files = []
    #os.listdir lists all the files in a given directory, and we just move through this list one by one
    # using the for loop structure we are familiar with
    for f in os.listdir('data/jokull'):
        #we just add this check to make sure we only readin files that end with .xy
        if f.endswith('.xy'):
            #then we add the file name to our list, making sure to give its full location with
            # respect to where we are currently executing code (i.e., we include 'data/jokull')
            files.append('data/jokull/'+f)
            
    #now we have a list of all the files we would like to read them in, create a polygon object
    # and loop over all the input file names
    #we will store our polygones in a list called 'polys'
    polys = []
    for f in files:
        #readin using pandas, you might have noticed that the jokull files have a first line
        # starting with '>', this 'header' needs to be skipped, so we tell pandas that it is a 'comment'
        # so that it gets ignored.
        dxy = pd.read_csv(f, comment='>', sep=' ', header=None)

        #now we use the shapely function to create a new polygon and add this to our list of polygons
        # the lon and lat data needs to be passed separately, but 'zipped' together
        polys.append(Polygon(zip(dxy[0], dxy[1])))
    
    #finally, we send our list of polygons back
    return polys

Now we have written our function, let's call it, obtain our list of polygon objects, and then plot them.

In [ ]:
#call our new function to obtain all the icesheet locations.
poly_geoms = poly_jokull()

#----------- same as before -----------------------
fg = plt.figure(figsize=(12, 6))
ax = fg.add_subplot(1, 1, 1,
                         projection=ccrs.Mercator(central_longitude=-15))
ax.set_extent([-25, -13, 63, 67.], crs=ccrs.Geodetic())

feature = ctp.feature.NaturalEarthFeature(
    name='coastline', category='physical',
    scale='10m',
    edgecolor='#000000', facecolor='lightgrey')
ax.add_feature(feature)

gl = ax.gridlines(draw_labels=True, linestyle=':', color='k', zorder=-1);
gl.xlabels_top = False
gl.ylabels_right = False
#----------- end of same as before -----------------------

#now, we need to loop over all of those polygons plotting them, which we do with the
# 'add_geometries' method of our axes object
# note, that here we need to use PlateCarree() as the coordinate reference system.
# and we set 'facecolor=white', because icesheets are white.
for p in poly_geoms:
    x = ax.add_geometries([p], crs=ccrs.PlateCarree(), facecolor='white')

### 4.2.3 Advanced file readin

Finally, let's add Iceland's volcanic rift zones.  This is a trickier problem.  We have been lucky so far with the files we have had to read in: they have all had a nice format that Pandas can accept.  Our next task is the first where the file needs a bit more pre-processing before it is useful to us.  Unfortunately, needing to pre-process input files is _very_ common!  

We could do this manually, but it is useful to see how to use some of the lower level file readin machinery in Python, because when you have a large file you don't want to be having to go through all of it yourself.  Potentially then having to repeat the exercise if the input file gets updated...

Like with the icesheets, we are going to define a function to do the readin and conversion to polygons for us.  Start by opening up the `fisswarms_fil.xy` file, so you can see the problems we face with this file. (Perhaps even try reading it in using Pandas to see what happens).

In [ ]:
#this gives us regular expression formatting, allowing us to identify words or patterns
# of letters in input.  You will see this in action below
import re
import pandas as pd
from shapely.geometry import Polygon

#define our function
# this time we are going to pass an argument, which is the file name
def poly_rift(file):

    #the 'with' statement here is useful for us, because it automatically closes the file
    # when we exit the statement.  It is saying 'with the file open, do some things, then close'
    # Like a for loop, everything within the 'with' statement needs to be properly indented.
    with open(file) as f:

        #we are going to store all our separate fissures initially as pandas dataframes
        # so let's create the list we are going to put them into here.
        dfs = []

        #once we have the file open, we then need to loop over lines in the file
        # we will use i to keep track of where we are, mainly to know whether this is the 
        # very first line of the file that we have read in
        for i, l in enumerate(f):

        #if you have looked in the file, you would have seen that the problem we have is
        # that the separate fissure swarms are separated by '>' indicators, so we need to 
        # split the file on that 'flag'. To do that, we need to perform a test of whether the line
        # we have read in starts with '>', we can do this using the 'regular expressions' library in Python
        # 're'.  So, we ask 'does the line, l, that we have just read in start with >?', which communicated 
        # as a regular expression reads '^>', where the '^' symbol indicates the start of the line
            if re.match('^>',l):
                #now, if there is a match AND this is NOT the first line we have read in,
                # then we want to store all the previous lines, as they represent a single fissure
                if i != 0:
                    #we use 'pd.DataFrame' to convert our list of lines into a data frame with two columns
                    dfs.append(pd.DataFrame(d, columns=('longitude','latitude')))
                    #convert all values of dataframe to floats i.e. decimal numbers rather than 'strings'
                    dfs[-1] = dfs[-1].astype('float64')
                #reset our list to store the lines representing lon lat coordinates of a given fissure
                d = []
            else:
                #this is the bit of code that gets executed when we read in a 'normal' data row
                #now we have the hardwork of converting the line we have read in into something we can store
                # in addition to having lines with '>' characters, this input file has multiple spaces before
                # and between the data.  We want to drop all of this, and just be left with the numbers.  
                # Let's step through what we need to do to clean up this input (in practice, we probably would want to
                # combine this all into one line of code)
                #1. remove the trailing newline character (each line has a character saying 'line end', we don't want to store this)
                l1 = l.rstrip('\n')
                #2. remove all the spaces at the beginning of the file, again we use ^ to indicate file beginning
                #   the '+' is saying 'find one or more instances of X' (where X in this case is just a space)
                #   the `re.sub(a, b, s)` command subtitutes 'a', for 'b' in the text string 's'.
                l2 = re.sub('^ +', '', l1)
                #3. now, let's remove all the repeated spaces in the line and leave just one space in each place
                l3 = re.sub(' +', ' ', l2)
                #4. we now have simple lines of numbers, so we just want to split these on the, now, single space
                #   separating them, we just use the '.split()' method that strings have, to split on a particular
                #   character
                l4 = l3.split(' ')
                
                #finally, we append the line to our list of lines for the current fissure
                # ...dropping the final entry as that is just 0's, so we use our list slicing syntax again
                # where we recall that '-1' indicates the end, which isn't included in the slice by default
                d.append(l4[0:-1])

        #do not forget this step!  The final lines read in need storing,
        # they won't be stored via previous dfs.append statement because it only
        # triggers when a new header is read in and the last lot of data isn't followed by another header (>)
        # it is followed by the end of the file
        dfs.append(pd.DataFrame(d, columns=('longitude','latitude')))
        dfs[-1] = dfs[-1].astype('float64')

    #now, we have all the data read in, we can convert these to polygons so we can plot them
    # this is the same was we did for the glaciers
    polys = []
    for d in dfs:
        polys.append(Polygon(zip(d.longitude, d.latitude)))

    return polys


Now that we have our function to readin the `fisswarms_fil.xy` file, let's run it and plot the fissures, copying our other plotting code from previously.

In [ ]:
#----------- same as before -----------------------
poly_geoms = poly_jokull()

fg = plt.figure(figsize=(12, 6))
ax = fg.add_subplot(1, 1, 1,
                         projection=ccrs.Mercator(central_longitude=-15))
ax.set_extent([-25, -13, 63, 67.], crs=ccrs.Geodetic())

feature = ctp.feature.NaturalEarthFeature(
    name='coastline', category='physical',
    scale='10m',
    edgecolor='#000000', facecolor='lightgrey')
ax.add_feature(feature)

gl = ax.gridlines(draw_labels=True, linestyle=':', color='k', zorder=-1);
gl.xlabels_top = False
gl.ylabels_right = False
#----------- end of same as before -----------------------

#let's update the icesheets to have a zorder of 2, so they plot on top of the rifts
for p in poly_geoms:
    x = ax.add_geometries([p], crs=ccrs.PlateCarree(), facecolor='white', zorder=2)
    
# run our new function
rift_polys = poly_rift('data/fisswarms_fil.xy')

#as with the ice sheets, we loop through each polygon, setting zorder=1 so they plot below the icesheets
# setting the color to that of fresh lava, 'rosybrown'
for p in rift_polys:
    x = ax.add_geometries([p], crs=ccrs.PlateCarree(), facecolor='rosybrown', zorder=1)

## 4.3 Plotting spatial data
***

Now we have our Iceland plot let's add the Holocene eruptions onto it.

In [ ]:
#setup the plot again...
# this time let's store all this in a function so we can skip writing this code each time.
def simp_plot():
    fg = plt.figure(figsize=(12, 6))
    ax = fg.add_subplot(1, 1, 1,
                             projection=ccrs.Mercator(central_longitude=-15))
    ax.set_extent([-25, -13, 63, 67.], crs=ccrs.Geodetic())

    feature = ctp.feature.NaturalEarthFeature(
        name='coastline', category='physical',
        scale='10m',
        edgecolor='#000000', facecolor='lightgrey')
    ax.add_feature(feature)

    gl = ax.gridlines(draw_labels=True, linestyle=':', color='k', zorder=-1);
    gl.xlabels_top = False
    gl.ylabels_right = False

    poly_geoms = poly_jokull()

    for p in poly_geoms:
        x = ax.add_geometries([p], crs=ccrs.PlateCarree(), facecolor='white', zorder=2)

    rift_polys = poly_rift('data/fisswarms_fil.xy')
    for p in rift_polys:
        x = ax.add_geometries([p], crs=ccrs.PlateCarree(), facecolor='rosybrown', zorder=1)

    #we want to return the figure, axis, and gridlin objects back from our function, 
    # in case we want to update their appearance later.
    return fg, ax, gl
        
#now we call our function to plot, and store the figure and axis objects that it returns
fg, ax, gl = simp_plot()

#now, let's add our data points and use some of the filtering we applied in Exercise 2
df = pd.read_excel('data/iceland_eruptions.xls')
#this next command gets us entries with: eruption names, longs, and lats. 
# Then we reduce to unique eruptions and take the mean of all the numerical properties of the dataframe
df_clean = df[(df.Eruption.notna()) & (df.Longitude.notna()) & (df.Latitude.notna())].groupby('Eruption').mean()

#let's plot this information
# notice, that when using 'ax.plot()' we need to specify the coordinate system using 'transform', rather than 'crs'
ax.plot(df_clean.Longitude, df_clean.Latitude, 'o', transform=ccrs.Geodetic());

Ok, that tells us where the eruptions are, but it would be nice to get some more information on the plot.  Let's do that by making the symbol size of the eruption scale with the eruption's volume.

We will also add a legend as a key to how the symbol size relates to eruption volume.  We will need to do this by manually setting the size of the symbol in the legend.  It is a bit of a messy solution, but does the job.

In [ ]:
import matplotlib.ticker as mticker

fg, ax, gl = simp_plot()

df = pd.read_excel('data/iceland_eruptions.xls')
df_clean = df[(df.Eruption.notna()) & (df.Longitude.notna()) & (df.Latitude.notna())].groupby('Eruption').mean()

#for convenience, let's this time create a new object that just contains the eruptions with volume information
dfvs = df_clean[df_clean['Volume(km3)'].notna()]
#let's be a bit obsessive: it would be good to get the small eruptions to plot last, so 
# they aren't obscurred by the large eruptions.  There are two options for this, we can use the 'zorder'
# parameter to explictly control the plotting order of objects when we use matplotlib to plot things, 
# or it might be easier to just order the dataframe itself.  We are going to do the latter as Pandas makes this
# easy
dfvs = dfvs.sort_values('Volume(km3)', ascending=False)

#let's plot this information, but scaling symbol size by eruption size
# to do that we are going to have to loop over each eruption because we can't just pass a long
# list of sizes to the plotting code and expect it to match up the sizes with the input data.
# Fortunately for us, eruption size in km3 translates quite nicely to symbol size in pt, 
# so we don't need to transform the raw volumes
# ...also, we need to explicitly turn line plotting off, so that all the points aren't connected up, hence
# 'linestyle='None''
for i, e in enumerate(dfvs['Volume(km3)']):
    #calculate marker size
    sz = dfvs['Volume(km3)'].iloc[i]
    ax.plot(dfvs.Longitude.iloc[i], dfvs.Latitude.iloc[i], marker='o', markerfacecolor='orangered',
            markeredgecolor='white', markersize=sz, linestyle='None',
            transform=ccrs.PlateCarree());

#let's also update the gridline locations, because the auto spacing is a bit useless
# ...an interval of 1.5 degrees in longitude!?  Bizarre.
gl.xlocator = mticker.MultipleLocator(1)
gl.ylocator = mticker.MultipleLocator(0.5)

#legend
#for this we need to manually access the plotting elements that allows 
# us to create circles, which we do by importing 'Line2D'
from matplotlib.lines import Line2D

#now, we create a list of symbols using Line2D.  Look up the manual page for Line2D to see all
# its functionality.  
# For now we are concerned in setting the marker as 'o', i.e., a circle; 
# setting the line style, ls, to '', which be leaving it empty is communicating we don't want a line drawn
legend_elements = [Line2D([0], [0], marker='o', ls = '', color='w', label='1 km$^3$', markeredgecolor='white',
                          markerfacecolor='orangered', markersize=1.),
                  Line2D([0], [0], marker='o', ls = '', color='w', label='10 km$^3$', markeredgecolor='white',
                          markerfacecolor='orangered', markersize=10),
                  Line2D([0], [0], marker='o', ls = '', color='w', label='20 km$^3$', markeredgecolor='white',
                          markerfacecolor='orangered', markersize=20),
                  Line2D([0], [0], marker='o', ls = '', color='w', label='30 km$^3$', markeredgecolor='white',
                          markerfacecolor='orangered', markersize=30)]

ax.legend(handles=legend_elements, loc='right')

#final finishing touch
ax.set_title('Iceland Holocene eruptions');

# Independent coding
***

<div class=obj>
    <b>Aim:</b> To readin and plot earthquake data from 2019 on the continental USA.
</div>
<p></p>

Often one of the first tasks when you have a geophysical or geochemical dataset is to get a sense of where your observations are located spatially.  Are there gaps in the data coverage?  Biases in locations that have been sampled?  All this can be quickly evaluated with a map.  Here we are going to look at earthquake locations from the continental US, __[downloaded from the USGS](https://earthquake.usgs.gov/earthquakes/search/)__, and stored in the data folder as `us_eqs.csv`.  

Some of the 'to-do's' in this exercise go a little beyond the code examples presented above.  This is a good opportunity to hone your search engine skills to find the answer online, pointers are also given to a couple of online examples that will help.

<p></p>

You should:
1. Create a new notebook, called `Exercise3_solution`.<br>
2. Readin the datafile `us_eqs.csv`, identifying the relevant columns for plotting.
3. Make a plot of the continental USA using the Albers Equal Area projection. (_Hint: __[Cartopy projections](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html)__)_.
4. Plot on the state boundaries (_Hint: always make good use of __[online examples](https://scitools.org.uk/cartopy/docs/latest/gallery/hurricane_katrina.html#sphx-glr-gallery-hurricane-katrina-py)__ when writing your code_.)
5. Plot on the earthquake locations.

Once you have the earthquake's plotted, consider why they are distributed where they are.  A good place to start thinking about this is on the __[USGS website](https://www.usgs.gov/natural-hazards/earthquake-hazards/hazards)__.

### Plot optimisattion

This gives us the basic plot, but we can try a few other things (see the hints below for some pointers if you are stuck).

- Can you color the point by magnitude, scale the point by depth?
- ...plotting the largest magnitude earthquakes first so they don't obscure the smaller events,
- ...and provide a scale bar for the depth color scale?

_Hint 1: If you want to iterate through a dataframe you can use `df.iterrows()`, as in `for d in df.iterrows():`, which will provide an object `d` containing the contents of each row, accessible as `d[1].mag`, for example._

_Hint 2: Specifying the colors for the points is a little trickier now, we need to chose a color map, i.e., __[one of these](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html)__, and then create a function to normalise our input data between 0 and 1, so it can pick a color off of the colormap.  Here is a code snippet to help_
```Python
    #we need some new imports
    import matplotlib.colors as clrs
    from matplotlib import cm

    #we use the min() and max() methods of the pandas dataframe to identify the limits of our data
    # this just gives us a function norm() we can pass data to and it rescales it to between 0 and 1
    norm = clrs.Normalize(df.depth.min(), df.depth.max())
    #here we chose our colormap
    cmap = cm.get_cmap('jet')
```
With the above code, we can then use `..., color=cmap(norm(color))...` within our plotting command to pick the color from our color map.

_Hint 3: Look __[here](https://stackoverflow.com/questions/41291534/make-colorbar-legend-in-matplotlib-cartopy/41587344)__ for how you can include a color scale on the plot._